In [5]:
!wget https://lang.org.ua/static/downloads/models/ubercorpus.cased.tokenized.word2vec.300d.bz2


--2025-06-26 19:29:12--  https://lang.org.ua/static/downloads/models/ubercorpus.cased.tokenized.word2vec.300d.bz2
Resolving lang.org.ua (lang.org.ua)... 65.21.91.242
Connecting to lang.org.ua (lang.org.ua)|65.21.91.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 555203899 (529M) [application/octet-stream]
Saving to: ‘ubercorpus.cased.tokenized.word2vec.300d.bz2’

ubercorpus.cased.to 100%[===================>] 529.48M  24.6MB/s    in 22s     

2025-06-26 19:29:36 (23.6 MB/s) - ‘ubercorpus.cased.tokenized.word2vec.300d.bz2’ saved [555203899/555203899]



In [6]:
!bzip2 -dk ubercorpus.cased.tokenized.word2vec.300d.bz2

In [1]:
!pip install gensim pymorphy3 pymorphy3-dicts-uk scikit-learn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account a

In [1]:
import gensim
import numpy as np
import pymorphy3
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

print("Бібліотеки успішно імпортовано.")

Бібліотеки успішно імпортовано.


In [16]:
# Набір аналогій на тему родинних зв'язків
# ВИПРАВЛЕНИЙ набір аналогій
# Формат: (A, B, C, D) де B - A + C ≈ D
# Приклад: ('чоловік', 'жінка', 'син', 'дочка') -> жінка - чоловік + син = дочка (гендерна аналогія)
family_analogies_corrected = [
    ('чоловік', 'жінка', 'син', 'дочка'),
    ('батько', 'мати', 'хлопчик', 'дівчинка'),
    ('дідусь', 'бабуся', 'юнак', 'дівчина'),
    ('брат', 'сестра', 'онук', 'онука'),
    ('король', 'королева', 'принц', 'принцеса'), # Класична перевірочна аналогія
    ('дядько', 'тітка', 'племінник', 'племінниця')
]

print(f"Створено набір з {len(family_analogies)} аналогій.")

Створено набір з 6 аналогій.


In [17]:
# Шлях до вашого файлу з моделлю
model_path = 'ubercorpus.cased.tokenized.word2vec.300d.bz2'

print("Завантаження моделі... Це може зайняти деякий час.")
# Завантажуємо модель
# Якщо у вас мало пам'яті, можете додати параметр limit=200000 для завантаження частини моделі
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)
print("Модель успішно завантажено.")

# Ініціалізуємо pymorphy3 для нормалізації слів
morph = pymorphy3.MorphAnalyzer(lang='uk')

Завантаження моделі... Це може зайняти деякий час.
Модель успішно завантажено.


In [18]:
def cosine_similarity(vec1, vec2):
    """Обчислює косинусну подібність між двома векторами."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def euclidean_distance(vec1, vec2):
    """Обчислює евклідову відстань між двома векторами."""
    return np.linalg.norm(vec1 - vec2)

# Перевірка роботи функцій
word1 = 'мама'
word2 = 'мати'

if word1 in model and word2 in model:
    vec1 = model[word1]
    vec2 = model[word2]

    cos_sim = cosine_similarity(vec1, vec2)
    euc_dist = euclidean_distance(vec1, vec2)

    print(f"Косинусна подібність між '{word1}' та '{word2}': {cos_sim:.4f}")
    print(f"Евклідова відстань між '{word1}' та '{word2}': {euc_dist:.4f}")

Косинусна подібність між 'мама' та 'мати': 0.4095
Евклідова відстань між 'мама' та 'мати': 28.4254


In [19]:
def normalize_word(word, morph_analyzer):
    """Повертає нормальну форму слова."""
    return morph_analyzer.parse(word)[0].normal_form

def find_analogy(word_a, word_b, word_c, model, morph_analyzer):
    """
    Знаходить слово d для аналогії a:b::c:d (b - a + c = d).
    Наприклад: find_analogy('син', 'батько', 'дочка', model) -> 'мати'
    """
    # Нормалізуємо слова
    word_a_norm = normalize_word(word_a, morph_analyzer)
    word_b_norm = normalize_word(word_b, morph_analyzer)
    word_c_norm = normalize_word(word_c, morph_analyzer)

    # Перевіряємо, чи всі слова є в моделі
    for w in [word_a_norm, word_b_norm, word_c_norm]:
        if w not in model:
            # print(f"Слово '{w}' відсутнє в моделі.")
            return None

    # Використовуємо most_similar для пошуку аналогії
    # b - a + c
    most_similar = model.most_similar(positive=[word_b_norm, word_c_norm], negative=[word_a_norm], topn=1)

    return most_similar[0][0]

# Тестування функції
predicted_word = find_analogy('син', 'батько', 'дочка', model, morph)
print(f"Аналогія 'батько' - 'син' + 'дочка' = '{predicted_word}'")

Аналогія 'батько' - 'син' + 'дочка' = 'мама'


In [21]:
correct_predictions = 0
total_analogies = len(family_analogies)

print("\n--- Тестування аналогій ---")
for a, b, c, expected_d in family_analogies:
    # Нормалізуємо очікуваний результат для коректного порівняння
    expected_d_norm = normalize_word(expected_d, morph)

    predicted_d = find_analogy(a, b, c, model, morph)

    if predicted_d:
        predicted_d_norm = normalize_word(predicted_d, morph)
    else:
        predicted_d_norm = "СЛОВО ВІДСУТНЄ"

    is_correct = (predicted_d_norm == expected_d_norm)
    if is_correct:
        correct_predictions += 1

    result_str = "Правильно" if is_correct else "Неправильно"
    print(f"'{b}' - '{a}' + '{c}' = '{predicted_d_norm}' (очікувалося: '{expected_d_norm}') -> {result_str}")

# Розрахунок точності
accuracy = (correct_predictions / total_analogies) * 100
print("\n--- Результат ---")
print(f"Правильно знайдено: {correct_predictions} з {total_analogies}")
print(f"Точність (Accuracy): {accuracy:.2f}%")


--- Тестування аналогій ---
'чоловік' - 'хлопчик' + 'жінка' = 'людин' (очікувалося: 'дівчинка') -> Неправильно
'батько' - 'син' + 'мати' = 'презерватив' (очікувалося: 'дочка') -> Неправильно
'дідусь' - 'онук' + 'бабуся' = 'бабець' (очікувалося: 'онук') -> Неправильно
'чоловік' - 'юнак' + 'дружина' = 'донька' (очікувалося: 'дівчина') -> Неправильно
'брат' - 'племінник' + 'сестра' = 'сестричка' (очікувалося: 'племінниця') -> Неправильно
'дядько' - 'син' + 'тітка' = 'бабець' (очікувалося: 'дочка') -> Неправильно

--- Результат ---
Правильно знайдено: 0 з 6
Точність (Accuracy): 0.00%


In [22]:
# --- ДІАГНОСТИЧНИЙ БЛОК КОДУ ---

# Виберемо одну, логічно правильну аналогію
a, b, c, expected_d = ('батько', 'мати', 'син', 'дочка')

print(f"--- Діагностика аналогії: '{b}' - '{a}' + '{c}' ≈ '{expected_d}' ---")

# Крок 1: Перевіряємо наявність слів у моделі
print("\nКрок 1: Перевірка наявності слів у моделі")
words_to_check = [a, b, c, expected_d]
all_words_present = True
for word in words_to_check:
    if word in model:
        print(f"Слово '{word}' ПРИСУТНЄ в моделі.")
    else:
        print(f"!!! Слово '{word}' ВІДСУТНЄ в моделі. Це може бути причиною помилки.")
        all_words_present = False

if not all_words_present:
    print("\nДіагностику зупинено, оскільки не всі слова є у словнику моделі.")
else:
    # Крок 2: Перевіряємо базову семантичну близькість
    print("\nКрок 2: Перевірка базової семантичної близькості (косинусна подібність)")
    print(f"Подібність 'мати' і 'дочка': {model.similarity('мати', 'дочка'):.4f}")
    print(f"Подібність 'батько' і 'син': {model.similarity('батько', 'син'):.4f}")
    print(f"Подібність 'мати' і 'батько': {model.similarity('мати', 'батько'):.4f}")
    print(f"Подібність 'дочка' і 'презерватив': {model.similarity('дочка', 'презерватив'):.4f}") # Порівняємо з дивним результатом

    # Крок 3: Обчислюємо вектор аналогії вручну
    print("\nКрок 3: Обчислення результуючого вектора вручну")
    try:
        # Векторна операція: B - A + C
        target_vector = model[b] - model[a] + model[c]
        print("Вектор аналогії успішно розраховано.")

        # Крок 4: Шукаємо найближчі слова до нашого вектора
        print(f"\nКрок 4: 10 найближчих слів до результуючого вектора ('{b}' - '{a}' + '{c}')")
        # Використовуємо most_similar з розрахованим вектором
        similar_words = model.most_similar(positive=[target_vector], topn=10)

        for i, (word, score) in enumerate(similar_words):
            print(f"{i+1}. '{word}' (подібність: {score:.4f})")

    except KeyError as e:
        print(f"Помилка при доступі до слова: {e}. Перевірте, чи правильно воно написане.")

--- Діагностика аналогії: 'мати' - 'батько' + 'син' ≈ 'дочка' ---

Крок 1: Перевірка наявності слів у моделі
Слово 'батько' ПРИСУТНЄ в моделі.
Слово 'мати' ПРИСУТНЄ в моделі.
Слово 'син' ПРИСУТНЄ в моделі.
Слово 'дочка' ПРИСУТНЄ в моделі.

Крок 2: Перевірка базової семантичної близькості (косинусна подібність)
Подібність 'мати' і 'дочка': 0.2188
Подібність 'батько' і 'син': 0.7673
Подібність 'мати' і 'батько': 0.1343
Подібність 'дочка' і 'презерватив': -0.0088

Крок 3: Обчислення результуючого вектора вручну
Вектор аналогії успішно розраховано.

Крок 4: 10 найближчих слів до результуючого вектора ('мати' - 'батько' + 'син')
1. 'мати' (подібність: 0.8456)
2. 'отримати' (подібність: 0.5795)
3. 'одержати' (подібність: 0.5500)
4. 'успадковувати' (подібність: 0.5468)
5. 'успадкувати' (подібність: 0.5364)
6. 'отримувати' (подібність: 0.5325)
7. 'носити' (подібність: 0.5264)
8. 'бачити' (подібність: 0.5204)
9. 'утворювати' (подібність: 0.5182)
10. 'втратити' (подібність: 0.5160)
